## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Nesta base de dados, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [10]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data

array([[[ -2.8834722 ,  18.218252  , -18.910511  , ...,   2.6398966 ,
          -3.7349935 ,   7.4631076 ],
        [  6.617178  ,  -3.8209395 ,   1.8515847 , ...,  -5.854906  ,
          -3.8280346 ,  -5.4327493 ],
        [-11.595966  ,  14.74045   ,   7.7698226 , ...,  -4.5176864 ,
           0.37455332,   6.6601686 ],
        [ 15.115495  , -15.170641  ,  -6.1311893 , ...,  16.025515  ,
          -0.93595564,   3.3250694 ]],

       [[  0.65565866,  -8.753227  ,   2.041236  , ...,  -7.0834394 ,
          12.440379  ,   4.9941688 ],
        [ -5.20613   ,  24.6411    ,  12.934999  , ...,  -3.7961366 ,
           0.11440572,  -6.7707815 ],
        [  6.6156116 ,  -8.973991  ,  10.461727  , ...,  -9.04715   ,
           4.6981435 ,  15.723002  ],
        [  8.303327  ,  -2.449911  ,  -8.16138   , ...,  12.354369  ,
          -1.4501842 ,  -4.5641375 ]],

       [[ -8.616586  ,  17.547148  ,  -0.64185965, ...,   1.3506331 ,
          15.566928  ,  11.141122  ],
        [ -5.6901703 ,  

In [19]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).